In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np

import analyze

In [ ]:
analyze.plot_sim_and_af(
    sim_dir=Path.cwd() / "src" / "sim" / "antenna_array",
    freq=2.45e9,
    xns=[1, 2, 4],
    yn=1,
    dxs=[60, 90],
    figname="antenna_array_comparison",
)